# Introduction

### Project initialization and setup

Importing all of the libraries that will be used. In the project.

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Connecting the DB to SQL

In [ ]:
DB_PATH = "viewer_interactions.db"

try:
    conn = sqlite3.connect(DB_PATH)
    print("Connected successfully!")
except sqlite3.Error as e:
    print("Connection failed:", e)

In [ ]:
# Display options
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

Listing all the tables

In [ ]:
tables_query = """
               SELECT name
               FROM sqlite_master
               WHERE type='table'
               ORDER BY name; \
               """

tables_df = pd.read_sql_query(tables_query, conn)
print("Tables in the database:")
display(tables_df)

Creating a dictionary of type table_name -> DataFrame

In [ ]:
table_names = tables_df["name"].tolist()

schemas = {}

for table in table_names:
    pragma_query = f"PRAGMA table_info({table});"
    schema_df = pd.read_sql_query(pragma_query, conn)
    schemas[table] = schema_df
    print(f"\nSchema for table '{table}':")
    display(schema_df)

implementing dfs and importing data

In [ ]:
dfs = {} # dictionary that maps the name of the table to the related data frame
# isnt it simpler if we just use universal dataframes for each dataset instead of having to worry about dictionaries

for t in table_names:
    df = pd.read_sql_query(f"SELECT * FROM {t};", conn)
    dfs[t] = df
    print(f"\nLoaded table '{t}' with shape {df.shape}")

# initializing the dataframes
movies_stats = dfs['movie_statistics']
movies = dfs['movies']
user_stats = dfs['user_statistics']
viewer_ratings = dfs['viewer_ratings']

### Methods

In [ ]:
# Refreshes one or more global dataframes by passing the actual df objects.
def sync_dataframe(*dfs_to_refresh):
    global movies_stats, movies, user_stats, viewer_ratings

    for dataframe in dfs_to_refresh:
        if dataframe is movies_stats:
            movies_stats = dfs["movie_statistics"]
        elif dataframe is movies:
            movies = dfs["movies"]
        elif dataframe is user_stats:
            user_stats = dfs["user_statistics"]
        elif dataframe is viewer_ratings:
            viewer_ratings = dfs["viewer_ratings"]
        else:
            raise ValueError("Unknown dataframe passed!")

Searching function

In [ ]:
# takes the name of the table, the name of the target key, and the value as inputs. Returns singular or multiple dataframes based on the request.

# can also search for null values if value is set to None

def search_by_parameter(table_name, key, value):
    df = dfs[table_name]

    if value is None:
        return df[df[key].isna()]

    return df[df[key] == value]

### Diagnostics

Counting all missing values, diagnostics purposes only

In [ ]:
for name, df in dfs.items():
    print(f"\n{name} missing values (%):")
    missing_pct = df.isna().mean() * 100
    display(missing_pct.to_frame("missing_%"))

### Movie Statistics Calculations

A function to calculate missing std. ratings of films

In [ ]:
def compute_film_std(df):
    sync_dataframe(df)

    film_stats = (
        df.groupby('movie_id')['rating']
        .apply(list)
        .reset_index(name='ratings')
    )

    def manual_std(ratings):
        ratings = np.array(ratings)
        n = len(ratings)
        if n <= 1:
            return 0.0
        mean = ratings.mean()
        return np.sqrt(((ratings - mean) ** 2).mean())

    film_stats['std_rating'] = film_stats['ratings'].apply(manual_std)

    return film_stats[['movie_id', 'std_rating']]

# Compute std for all films
viewer_ratings = dfs['viewer_ratings']
film_std = compute_film_std(viewer_ratings)

# Load movies_statistics
movies_stats = dfs["movie_statistics"]

# Compute the old percentage before merging
old_null_pct = dfs["movie_statistics"]["std_rating"].isna().mean() * 100

# Merge new std values
movies_stats = movies_stats.merge(
    film_std,
    on="movie_id",
    how="left",
    suffixes=("", "_new")
)

# Replace old std_rating with the new one
movies_stats["std_rating"] = movies_stats["std_rating_new"]
movies_stats.drop(columns=["std_rating_new"], inplace=True)

# Save updated table
dfs["movie_statistics"] = movies_stats

# Compute new percentage ---
new_null_pct = movies_stats["std_rating"].isna().mean() * 100

# absolute improvement (percentage points)
improvement_abs = old_null_pct - new_null_pct

# relative improvement (how many percent of the original NaNs we removed)
improvement_rel = (improvement_abs / old_null_pct) * 100 if old_null_pct > 0 else 0

print(f"Missing values reduced from {old_null_pct:.2f}% to {new_null_pct:.2f}%.")
print(f"Absolute improvement: {improvement_abs:.2f}%")
print(f"Relative improvement: {improvement_rel:.2f}% better than before.")

In [ ]:
#Figure out how to drop na values in general
movies_stats = dfs['movie_statistics']

print(f"Before cleaning: {len(movies_stats)} movies")
movies_stats = movies_stats.dropna(subset=['std_rating'])
dfs['movie_statistics'] = movies_stats
print(f"After removing single-rating movies: {len(movies_stats)} movies")

Calculating the missing total_ratings of movies

In [ ]:
sync_dataframe(movies_stats)

# Collecting all the movies with absent total_rating in a dictionary
missing_dict = {}

missing = search_by_parameter('movie_statistics', 'total_ratings', None)
missing_dict = {row.movie_id: 0 for row in missing.itertuples(index=False)}

# Iterating through viewer_ratings and manually counting the ratings for each film
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id
    if movie_id in missing_dict:
        missing_dict[movie_id] += 1

# Update movie_stats
for row in movies_stats.itertuples(index=True):
    if row.movie_id in missing_dict:
        movies_stats.at[row.Index, "total_ratings"] = missing_dict[row.movie_id]

dfs["movie_statistics"] = movies_stats

Calculating Missing Averages

In [ ]:
sync_dataframe(movies_stats, viewer_ratings)

# Finding movies with null avg_rating
missing_avg = search_by_parameter('movie_statistics', 'avg_rating', None)

# Creating a dict of type { movie_id : avg_rating }
# set 0 as base value for now, might change it later
missing_avg_dict = {row.movie_id: 0 for row in missing_avg.itertuples(index=False)}

# Storing the sum of all ratings for each movie
rating_sums = {movie_id: 0 for movie_id in missing_avg_dict}

# Iterating through viewer ratings and adding to sum if movie_id matches
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id
    rating = row.rating

    if movie_id in missing_avg_dict:
        rating_sums[movie_id] += rating

for row in movies_stats.itertuples():
    movie_id = row.movie_id

    if movie_id in rating_sums:
        total = row.total_ratings # I'm assuming that my calculations of total_ratings per movie is correct ang i got rid of                            all null values

        # IF FORE SOME MAGICAL REASON THERE IS STILL A NULL THEN IGNORE
        if pd.isna(total) or total == 0:
            avg = 0
        else:
            avg = rating_sums[movie_id] / total

        movies_stats.at[row.Index, "avg_rating"] = avg

Calculating the missing min and max ratings for movies

In [ ]:
sync_dataframe(movies_stats)

# Find movies with missing min_rating and max_rating using your function
missing_min = search_by_parameter('movie_statistics', 'min_rating', None)
missing_max = search_by_parameter('movie_statistics', 'max_rating', None)

# Combine them as some movies may be in both
missing_ids = set(missing_min["movie_id"]) | set(missing_max["movie_id"])

# Take only ratings for the movies we care
relevant_ratings = viewer_ratings[viewer_ratings["movie_id"].isin(missing_ids)]

# Building a nested dict {movie_id : {"min": ..., "max": ...}}
min_max_dict = {}

for row in relevant_ratings.itertuples(index=False):
    movie_id = row.movie_id
    rating = row.rating

    if movie_id not in min_max_dict:
        min_max_dict[movie_id] = {"min": rating, "max": rating}
    else:
        if rating < min_max_dict[movie_id]["min"]:
            min_max_dict[movie_id]["min"] = rating
        if rating > min_max_dict[movie_id]["max"]:
            min_max_dict[movie_id]["max"] = rating

# Update movie_statistics
for row in movies_stats.itertuples(index=True):
    movie_id = row.movie_id

    if movie_id in min_max_dict:
        if pd.isna(row.min_rating):
            movies_stats.at[row.Index, "min_rating"] = min_max_dict[movie_id]["min"]
        if pd.isna(row.max_rating):
            movies_stats.at[row.Index, "max_rating"] = min_max_dict[movie_id]["max"]

dfs["movie_statistics"] = movies_stats

Finding missing unique users

In [ ]:
sync_dataframe(movies_stats)

missing_unique = search_by_parameter('movie_statistics', 'unique_users', None)

# creating my favorite movie set
missing_movie_ids = {row.movie_id for row in missing_unique.itertuples(index=False)}

# creating a dict movie_id: customer_id
unique_users_dict = {movie_id: set() for movie_id in missing_movie_ids}

# Gathering unique users
for row in viewer_ratings.itertuples(index=False):
    movie_id = row.movie_id

    # as always im getting only those movies which have null for unique users
    if movie_id in unique_users_dict:
        unique_users_dict[movie_id].add(row.customer_id)

# Counting unique users
updated = 0
for movie_id, users in unique_users_dict.items():
    count = len(users)  # unique users count

    movies_stats.loc[
        movies_stats["movie_id"] == movie_id,
        "unique_users"
    ] = count

    updated += 1

# just in case if a movie has 0 ratings im setting unique users to 0
movies_stats["unique_users"] = movies_stats["unique_users"].fillna(0).astype(int)

# Updating
dfs["movie_statistics"] = movies_stats

Checking for any duplicate movie ids and removing them from all datasets to clean the data.

### User_statistics calculations

In [ ]:
#std

In [ ]:
#total

In [ ]:
#avg

### Merging datasets

Merging the Movies and movie statis filling in missing values on either dataset and converting all of the dates to type DateTime as well as all counts and years to integers.
This is in order to clean our movie data before merging it with our user data to fill in any recoverable missing values.
Rows with missing values will be dropped.

In [ ]:
movies = dfs['movies'].copy() # does this overwrite the movies we have right now? maybe call it something else.
movie_stats = dfs['movie_statistics'].copy()
viewer_ratings = dfs['viewer_ratings'].copy()

bad_ids_movies = movies.loc[movies['movie_id'].duplicated(keep=False), 'movie_id'].unique().tolist()
bad_ids_stats = movie_stats.loc[movie_stats['movie_id'].duplicated(keep=False), 'movie_id'].unique().tolist()
bad_ids = list(set(bad_ids_movies + bad_ids_stats))
print('bad movie ids to remove: ', bad_ids)

viewer_ratings = viewer_ratings[~viewer_ratings['movie_id'].isin(bad_ids)]
movies = movies[~movies['movie_id'].isin(bad_ids)]
movie_stats = movie_stats[~movie_stats['movie_id'].isin(bad_ids)]

movies['year_of_release'] = (pd.to_numeric(movies['year_of_release'], errors='coerce').astype('Int64'))
movie_stats['total_ratings'] = (pd.to_numeric(movie_stats['total_ratings'], errors='coerce').astype('Int64'))
movie_stats['unique_users'] = (pd.to_numeric(movie_stats['unique_users'], errors='coerce').astype('Int64'))
movie_stats['year_of_release'] = (pd.to_numeric(movie_stats['year_of_release'], errors='coerce').astype('Int64'))

movie_stats['first_rating_date'] = pd.to_datetime(movie_stats['first_rating_date'], errors='coerce')
movie_stats['last_rating_date'] = pd.to_datetime(movie_stats['last_rating_date'], errors='coerce')

# this will automatically remove rows with missing data
movie_full = movies.merge(movie_stats, on='movie_id', how='inner', suffixes=('_movies', '_stats'))

movie_full['title'] = movie_full['title_movies'].combine_first(movie_full['title_stats'])
movie_full['year_of_release'] = movie_full['year_of_release_movies'].combine_first(movie_full['year_of_release_stats'])

movie_full = movie_full.drop(columns=['title_movies', 'title_stats', 'year_of_release_movies', 'year_of_release_stats'])

dfs['movies'] = movies
dfs['movie_statistics'] = movie_stats
dfs['viewer_ratings'] = viewer_ratings
dfs['movie_full'] = movie_full

In [ ]:
print("movies dtypes:")
print(movies.dtypes)

print("\nmovie_stats dtypes:")
print(movie_stats.dtypes)

print("\nmovie_full dtypes:")
print(movie_full.dtypes)

- Converts the date parameter in viewer_ratings to datetime.
- Merges viewer_ratings, movies, movie_statistics and user_statistics into one dataset as merged_data.

In [ ]:
user_stats = dfs['user_statistics'].copy()
viewer_ratings = dfs['viewer_ratings'].copy()
movie_full = dfs['movie_full'].copy()

viewer_ratings['date'] = pd.to_datetime(viewer_ratings['date'], errors = 'coerce')
user_stats['first_rating_date'] = pd.to_datetime(user_stats['first_rating_date'], errors = 'coerce')
user_stats['last_rating_date'] = pd.to_datetime(user_stats['last_rating_date'], errors = 'coerce')

user_stats['total_ratings'] = (pd.to_numeric(user_stats['total_ratings'], errors='coerce').astype('Int64'))
user_stats['unique_movies'] = (pd.to_numeric(user_stats['unique_movies'], errors='coerce').astype('Int64'))
user_stats['activity_days'] = (pd.to_numeric(user_stats['activity_days'], errors='coerce').astype('Int64'))


# create user and movie specific columns
movie_full = movie_full.rename(columns={
    'total_ratings':     'movie_total_ratings',
    'avg_rating':        'movie_avg_rating',
    'std_rating':        'movie_std_rating',
    'min_rating':        'movie_min_rating',
    'max_rating':        'movie_max_rating',
    'first_rating_date': 'movie_first_rating_date',
    'last_rating_date':  'movie_last_rating_date'
})


user_stats = user_stats.rename(columns={
    'total_ratings':     'user_total_ratings',
    'avg_rating':        'user_avg_rating',
    'std_rating':        'user_std_rating',
    'min_rating':        'user_min_rating',
    'max_rating':        'user_max_rating',
    'first_rating_date': 'user_first_rating_date',
    'last_rating_date':  'user_last_rating_date'
})

# drop anomalous date for user stats
if 'anomalous_date' in viewer_ratings.columns:
    viewer_ratings = viewer_ratings.drop(columns=['anomalous_date'])

viewer_ratings.dtypes

merged_data = viewer_ratings.merge(movies, on = 'movie_id', how = 'left')
merged_data = merged_data.merge(user_stats, on = 'customer_id', how = 'left')
merged_data = merged_data.merge(movie_stats, on = 'movie_id', how = 'left')
# avg rating standard rating mean rating
'''
want to merge
- title
- year of release
want to keep independent
- avg rating
- std rating
- min rating
- max rating
- first rating date
- last rating date
'''

print("Total columns:", len(merged_data.columns))
list(merged_data.columns)

Giacomo

### Plotting Statistics and Overall findings

Ryder

In [ ]:
import seaborn as sns


# Markdown cell above
"""
### Global Rating Distribution

There is a clear over positive bias. This can be due to the fact that this was done on a streaming platform where users are more likely to rate higher
"""

plt.figure(figsize=(11, 6)) # made it this size for a better fit in the read me
ax = sns.histplot(
    data=dfs['viewer_ratings'],
    x='rating',
    bins=5,
    discrete=True, #tells seaborn to treat x as integer values
    color="#0062ff",
    edgecolor='white',
    alpha=0.85,  #transparency
    linewidth=1.5
)

# Add exact counts on top of each bar
for rect in ax.patches:
    height = rect.get_height()
    ax.text(
        rect.get_x() + rect.get_width()/2., 
        height + 200_000,                    # a bit above the bar
        f'{int(height):,}',                  # adds commas: 12,345,678
        ha='center', va='bottom', fontsize=12, fontweight='bold'
    )

# Clean y-axis
plt.ylabel('Number of Ratings (millions)', fontsize=12)
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1e6:.1f}M'))

plt.title('Distribution of Viewer Ratings\n(Strong Positive Bias)', 
          fontsize=15, pad=20)
plt.xlabel('Rating (1–5)', fontsize=12)
plt.xticks(range(0, 7))
plt.ylim(0, 2_000_000)
plt.show()

In [ ]:
"""
### User Average Rating vs. Rating Variability

Scatter from user_statistics: High avg with low std = consistent positive raters. 
Valuable for grouping users (e.g., strict critics vs. easy fans). Alpha for density.
"""

sync_dataframe(user_stats)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=user_stats, x='avg_rating', y='std_rating', alpha=0.4, color='#0062ff', edgecolor=None)
plt.title('User Average Rating vs. Standard Deviation\n(Rating Consistency Patterns)', fontsize=15, pad=20)
plt.xlabel('Average Rating Given (1–5)', fontsize=12)
plt.ylabel('Std Deviation of Ratings', fontsize=12)
plt.xlim(0, 5)
plt.ylim(0, 4)  # Std typically low due to bias
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# PERFECT PENTAGON — FINAL WORKING VERSION (Dec 2025)
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch

# CHANGE ONLY THESE TWO VALUES
avg_rating_value = user_stats['avg_rating'].mean()   # ← your real global avg from user_statistics
std_rating_value = user_stats['std_rating'].mean()   # ← your real global std from user_statistics

# Convert to 0–1 scale
pull_to_five = (avg_rating_value - 1.0) / 4.0
consistency  = np.clip(1.0 - (std_rating_value / 1.8), 0.1, 1.0)

# Pentagon vertices (5 corners + first repeated to close)
center = np.array([0.5, 0.5])
radius = 0.45
angles = np.linspace(0, 2*np.pi, 5, endpoint=False)
verts = center + radius * np.column_stack([np.cos(angles), np.sin(angles)])
verts = np.vstack([verts, verts[0]])  # close the loop → shape (6, 2)

# Blue pentagon (pulled toward Rating 5 = bottom-right)
blue_scale = 0.85 * pull_to_five
blue_verts = center + blue_scale * (verts - center)

# Red pentagon (size = consistency)
red_scale = 0.92 * consistency
red_verts = center + red_scale * (verts - center)

# Correct Path codes: 6 vertices → 6 codes
codes = [Path.MOVETO] + [Path.LINETO]*4 + [Path.CLOSEPOLY]

# Plot
fig, ax = plt.subplots(figsize=(10, 10))

# Outer frame
frame_path = Path(verts, codes)
ax.add_patch(PathPatch(frame_path, facecolor='none', edgecolor='black', linewidth=3))

# Red pentagon (consistency)
red_path = Path(red_verts, codes)
ax.add_patch(PathPatch(red_path, facecolor="#000000", alpha=0.7, edgecolor='#aa0000', linewidth=4))

# Blue pentagon (average rating)
blue_path = Path(blue_verts, codes)
ax.add_patch(PathPatch(blue_path, facecolor="#00ffdd", alpha=0.75, edgecolor='#003380', linewidth=4))

# Corner labels
labels = ['1', '2', '3', '4', '5']
for (x, y), label in zip(verts[:-1], labels):
    ax.text(x, y, label, fontsize=16, fontweight='bold', ha='center', va='center',
            bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.95))

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.axis('off')

ax.set_title(f"Global User Rating Personality Pentagon\n"
             f"Avg Rating = {avg_rating_value:.2f} | Std Dev = {std_rating_value:.2f}",
             fontsize=20, pad=40, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
'Plotting a timeline of ratings over time'

sync_dataframe(movies_stats)

plt.figure(figsize=(15, 10))
sns.scatterplot(data=movies_stats, x='year_of_release', y='avg_rating', alpha=0.4, color='#0062ff', edgecolor=None, s=35)
plt.title('Movie Rating VS. Year of Release\n(Trends Over Time)', fontsize=15, pad=20)
plt.xlabel('Year of Release', fontsize=12)
plt.ylabel('Average Rating Given (1–5)', fontsize=12)
plt.xlim(1890, 2010) #this included the data for all the movies in the data set as it goes from 1896 to 2005
plt.ylim(0, 7)  
#plt.scatter(x,y, s)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

### Work in progress/Miscellaneous 

Giacomo's Ting

In [ ]:
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';",
    conn
    )['name'].tolist()

print("=== DATA DICTIONARY ===\n")

for table in tables:
    print(f"Table: {table}")
    print("-" * (7 + len(table)))

    # Get actual column info from PRAGMA but filter to nice output
    schema = pd.read_sql(f"PRAGMA table_info('{table}')", conn)

    # Keep only real schema fields you want (remove cid, default, pk if desired)
    clean_schema = schema[['name', 'type', ]]

    print(clean_schema.to_string(index=False))
    print("\n")